In [1]:
#!/usr/bin/env python3 

import sys
import pandas as pd
import datetime
import requests
import numpy as np
from pandas import ExcelWriter
import os
import glob
import pytz
from dateutil.tz import gettz
import timeit
# from datetime import datetime
from datetime import timedelta
import time
#
#

In [12]:
sheets_dict = pd.read_excel('Validation.xlsx', sheet_name = None)

In [28]:

for name, sheet in sheets_dict.items():
#     df = pd.DataFrame()
    sheet['sheet'] = name
    sheet = sheet.rename(columns=lambda x: x.split('\n')[-1])
    df = pd.DataFrame(sheet)
    df = df[['MeterSerial','sheet']]
    df['Meter'] = df['MeterSerial'].astype(str).str[:14]
    
    
    print(df.head())

# full_table.reset_index(inplace=True, drop=True)


          MeterSerial       sheet           Meter
0      102.105.000150  Filellinon  102.105.000150
1  102.101.000103RFB1  Filellinon  102.101.000103
2      102.105.000046  Filellinon  102.105.000046
3      102.105.000061  Filellinon  102.105.000061
4      102.105.000065  Filellinon  102.105.000065
      MeterSerial    sheet           Meter
0  102.107.000006  Mosxato  102.107.000006
1  102.107.000135  Mosxato  102.107.000135
2  102.107.000006  Mosxato  102.107.000006
3  102.107.000017  Mosxato  102.107.000017
4  102.107.000135  Mosxato  102.107.000135
      MeterSerial     sheet           Meter
0  102.104.000065  N. Ionia  102.104.000065
1  102.107.000051  N. Ionia  102.107.000051
2  102.104.000023  N. Ionia  102.104.000023
3  102.104.000029  N. Ionia  102.104.000029
4  102.104.000036  N. Ionia  102.104.000036
          MeterSerial       sheet           Meter
0  102.101.000357RFB1  N. Psyxiko  102.101.000357
1  102.101.000373RFB1  N. Psyxiko  102.101.000373
2  102.101.000378RFB1  N. Ps

In [81]:
def conv_to_consumption(df, interval):
    #     convert cumulative energy to consumed energy
    if 'cnrgA' in df.columns:
        df['diffA'] = np.nan
        df.diffA[((df.cnrgA.isna() == False) & (df.cnrgA.shift().isna() == False))] = df.cnrgA - df.cnrgA.shift()
        df['diffA'] = df['diffA']/1000
        df.diffA.iloc[0] = 0
        df.rename(columns={"diffA": "Consumed energy (kWh) A"}, inplace = True)
        df.drop(['cnrgA'], axis=1, inplace=True)

    if 'cnrgB' in df.columns:
        df['diffB'] = np.nan
        df.diffB[(df.cnrgB.isna() == False) & (df.cnrgB.shift().isna() == False)] = df.cnrgB - df.cnrgB.shift()
        df['diffB'] = df['diffB']/1000
        df.diffB.iloc[0] = 0
        df.rename(columns={"diffB": "Consumed energy (kWh) B"}, inplace=True)
        df.drop(['cnrgB'], axis=1, inplace=True)

    if 'cnrgC' in df.columns:
        df['diffC'] = np.nan
        df.diffC[(df.cnrgC.isna() == False) & (df.cnrgC.shift().isna() == False)] = df.cnrgC - df.cnrgC.shift()
        df['diffC'] = df['diffC']/1000
        df.diffC.iloc[0] = 0
        df.rename(columns={"diffC": "Consumed energy (kWh) C"}, inplace=True)
        df.drop(['cnrgC'], axis=1, inplace=True)

    if (('Consumed energy (kWh) A' in df.columns) and ('Consumed energy (kWh) B' in df.columns) and ('Consumed energy (kWh) C' in df.columns)):
        df['total'] = np.nan
        df.total[(df['Consumed energy (kWh) A'].isna() == False) & (df['Consumed energy (kWh) B'].isna() == False) & (
                df['Consumed energy (kWh) C'].isna() == False)] = df['Consumed energy (kWh) A'] + df['Consumed energy (kWh) B'] + df['Consumed energy (kWh) C']
        df.rename(columns={"total": "Total Consumed energy (kWh)"}, inplace = True)

    return df

def align_resample(df, interval):

    df.index = df.index.map(lambda x: x.replace(second=0, microsecond=0))
    df = df.groupby(df.index).max()
    df.sort_index(inplace=True)
    df = conv_to_consumption(df, interval)
    
    df = df.resample(interval+'T').sum().copy()
    df['ts'] = df.index
    
#     df = df.resample(interval+'T').mean()
#     df.reset_index(inplace = True, drop = False)
#     df.set_index('ts',inplace = True, drop = False)
#     df.drop(['Consumed energy (kWh) A','Consumed energy (kWh) B','Consumed energy (kWh) C'], axis=1, inplace=True)
#     df = pd.concat([df,df_nrg[['Consumed energy (kWh) A','Consumed energy (kWh) B','Consumed energy (kWh) C']]], axis = 1)
#     print('New DF',df.head())
   
    return df

In [90]:
import datetime

def read_data(devid, acc_token, address, start_time, end_time, interval, descriptors, tmzn):

    r2 = requests.get(
        url=address + "/api/plugins/telemetry/DEVICE/" + devid + "/values/timeseries?keys="+descriptors+"&startTs=" + start_time + "&endTs=" + end_time + "&agg=NONE&limit=1000000",
        headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()
    if r2:
        df = pd.DataFrame([])

        

        if 'cnrgA' in r2.keys():
            df13 = pd.DataFrame(r2['cnrgA'])
            df13.set_index('ts', inplace=True)
            df13.columns = ['cnrgA']
            df = pd.concat([df,df13], axis=1)

        if 'cnrgB' in r2.keys():
            df14 = pd.DataFrame(r2['cnrgB'])
            df14.set_index('ts', inplace=True)
            df14.columns = ['cnrgB']
            df = pd.concat([df,df14], axis=1)

        if 'cnrgC' in r2.keys():
            df15 = pd.DataFrame(r2['cnrgC'])
            df15.set_index('ts', inplace=True)
            df15.columns = ['cnrgC']
            df = pd.concat([df,df15], axis=1)

        

        if df.empty == False:
        
            df.reset_index(drop=False, inplace=True)
            df['ts'] = pd.to_datetime(df['ts'], unit='ms')
            # df['ts'] = df['ts'].dt.tz_localize('utc').dt.tz_convert(tmzn).dt.tz_localize(None)
    
            df = df.sort_values(by=['ts'])
            df.reset_index(drop=True, inplace=True)
            df.set_index('ts',inplace = True, drop = True)
            for col in df.columns:
                df[col] = df[col].astype('float')
    
            df = align_resample(df, interval)
    
            
            df['ts'] = df['ts'].dt.tz_localize('utc').dt.tz_convert(tmzn).dt.tz_convert(None)
            df['Year'] = df['ts'].dt.year
            df['month'] =  df['ts'].dt.month
#             print(df.head())
            
        else:
            df = pd.DataFrame([])
    else:
        df = pd.DataFrame([])
        print('Empty json!')
    return df








In [95]:

    
entityName = 'N.Psyxiko'

entityID = str('0b3407b0-07ac-11ea-a66e-7bc859fa1c1b')
start_time = '0'
end_time =  '1575158399000'
interval = '1440'
descriptors = 'cnrgA,cnrgB,cnrgC'
tmzn = 'Europe/Athens'

path = '../xlsx files'

path = path+'/'
os.chdir(path)
filename = entityName+'_'+start_time+'_'+end_time+'.xlsx'



# address = "http://localhost:8080/"
address =  "https://m3.zeb.gr"

r = requests.post(address + "/api/auth/login",
                      json={'username': 'a.papagiannaki@meazon.com', 'password': 'eurobank'}).json()


# acc_token is the token to be used in the next request
acc_token = 'Bearer' + ' ' + r['token']

r1 = requests.get(url=address + '/api/relations/info?fromId=' + entityID + '&fromType=ASSET',
                  headers={'Content-Type': 'application/json', 'Accept': '*/*',
                           'X-Authorization': acc_token}).json()
sum_nrg = pd.DataFrame([])
with pd.ExcelWriter(filename) as writer:
    for device in r1:

        # read ID and name of building's devices
        devid  = str(device['to']['id'])
        devName = str(device['toName'])
        
        print(devName)

        #print('devname:',devName)
        summary = read_data(devid,acc_token,address, start_time, end_time, interval, descriptors,tmzn)
        
#         if ('Total Consumed energy (kWh)' in summary.columns):
#             print('3phase meter')
#             new_row = {'Power meter':devName, 'Total consumed energy (kWh)':summary['Total Consumed energy (kWh)'].sum()}
# #             sum_nrg = sum_nrg.append(new_row, ignore_index = True)     
# #             sum_nrg.to_excel(writer,sheet_name = 'Summary', index = False)
#         elif ('Consumed energy (kWh) A' in summary.columns):
#             new_row = {'Power meter':devName, 'Total consumed energy':summary['Consumed energy (kWh) A'].sum()}
# #             sum_nrg = sum_nrg.append(new_row, ignore_index = True)  
# #             sum_nrg.to_excel(writer,sheet_name = 'Summary', index = False)
        if summary.empty==False:
            summary = summary[['Total Consumed energy (kWh)','Year','month']]
            cols = summary.columns.tolist()
            cols = cols[-2:]+cols[:-2]
            summary = summary[cols]
            summary['Year'] = summary['Year'].astype(str)
            summary['month'] = summary['month'].astype(str)
            summary['Year_Month'] = summary['Year'].str.cat(summary['month'], sep ="_") 
            grouped = pd.DataFrame(summary.groupby([(summary['Year_Month'])]).sum())
    #         grouped = pd.DataFrame(summary.groupby([(summary['Year']),(summary['month'])]).sum())
            print(grouped.head())
        
            devName = devName.replace(':','')
            grouped.to_excel(writer,sheet_name = devName, index = True)

writer.save()



102.105.000238
Empty json!
102.101.000631
            Total Consumed energy (kWh)
Year_Month                             
2017_11                        1136.081
2017_12                        2117.801
2018_1                         2715.536
2018_10                        1146.694
2018_11                        1322.506
102.101.000633
            Total Consumed energy (kWh)
Year_Month                             
2017_11                         543.633
2017_12                         470.745
2018_1                          496.685
2018_10                         600.280
2018_11                         501.055
102.101.000635
            Total Consumed energy (kWh)
Year_Month                             
2017_11                         331.825
2017_12                         351.879
2018_1                          363.051
2018_10                         532.907
2018_11                         579.746
102.101.000634
            Total Consumed energy (kWh)
Year_Month                       

In [99]:

    
entityName = 'Mosxato'

entityID = str('fe819a50-07ab-11ea-a66e-7bc859fa1c1b')
start_time = '0'
end_time =  '1575158399000'
interval = '1440'
descriptors = 'cnrgA,cnrgB,cnrgC'
tmzn = 'Europe/Athens'

path = '../xlsx files'

path = path+'/'
os.chdir(path)
filename = entityName+'_data.xlsx'



# address = "http://localhost:8080/"
address =  "https://m3.zeb.gr"

r = requests.post(address + "/api/auth/login",
                      json={'username': 'a.papagiannaki@meazon.com', 'password': 'eurobank'}).json()


# acc_token is the token to be used in the next request
acc_token = 'Bearer' + ' ' + r['token']

r1 = requests.get(url=address + '/api/relations/info?fromId=' + entityID + '&fromType=ASSET',
                  headers={'Content-Type': 'application/json', 'Accept': '*/*',
                           'X-Authorization': acc_token}).json()
sum_nrg = pd.DataFrame([])
with pd.ExcelWriter(filename) as writer:
    for device in r1:

        # read ID and name of building's devices
        devid  = str(device['to']['id'])
        devName = str(device['toName'])
        
        print(devName)

        #print('devname:',devName)
        summary = read_data(devid,acc_token,address, start_time, end_time, interval, descriptors,tmzn)
        
#         if ('Total Consumed energy (kWh)' in summary.columns):
#             print('3phase meter')
#             new_row = {'Power meter':devName, 'Total consumed energy (kWh)':summary['Total Consumed energy (kWh)'].sum()}
# #             sum_nrg = sum_nrg.append(new_row, ignore_index = True)     
# #             sum_nrg.to_excel(writer,sheet_name = 'Summary', index = False)
#         elif ('Consumed energy (kWh) A' in summary.columns):
#             new_row = {'Power meter':devName, 'Total consumed energy':summary['Consumed energy (kWh) A'].sum()}
# #             sum_nrg = sum_nrg.append(new_row, ignore_index = True)  
# #             sum_nrg.to_excel(writer,sheet_name = 'Summary', index = False)
        if summary.empty==False:
            summary = summary[['Total Consumed energy (kWh)','Year','month']]
            cols = summary.columns.tolist()
            cols = cols[-2:]+cols[:-2]
            summary = summary[cols]
#             summary['Year'] = summary['Year'].astype(str)
#             summary['month'] = summary['month'].astype(str)
#             summary['Year_Month'] = summary['Year'].str.cat(summary['month'], sep ="_") 
#             grouped = pd.DataFrame(summary.groupby([(summary['Year_Month'])]).sum())

        
            devName = devName.replace(':','')
            summary.to_excel(writer,sheet_name = devName, index = True)

writer.save()



102.116.000022
102.201.000070
102.201.000319
102.107.000024
102.107.000017
102.107.000135
102.107.000006
102.116.000047
102.116.000046
102.116.000045
102.116.000051
102.116.000061
102.116.000060
102.116.000069
102.116.000063
